In [ ]:
#
# Copyright 2023 Google LLC
# 
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
# 
#     https://www.apache.org/licenses/LICENSE-2.0
# 
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
#

# Deepspeed Chat on Vertex AI

NOTE: This is an example to test multi-mode training with DeepSpeed on Vertex. This folder is to use torchrun launcher to launcher the multi-process, not use default deepspeed launcher.

DeepspeedChat has 3 steps: SFT, Reward Model, and RLHF. We are only calling the SFT step here.

# Setup Container

In [ ]:
# Artifact Registry Repo
AR_REPO="llama2"
IMG_NAME="deepspeed-chat"
TAG="vertex-torchrun"

In [ ]:
PROJECT_ID="PROJECT_ID"
LOCATION="us-central1"
BUCKET="gs://BUCKET_NAME"
IMAGE_URI=f"{LOCATION}-docker.pkg.dev/{PROJECT_ID}/{AR_REPO}/{IMG_NAME}:{TAG}"

In [ ]:
%cd ~/vertex-deepspeed

/home/jupyter/vertex-deepspeed


In [ ]:
# build docker image
EXAMPLE_DIR="deepspeed-chat"
DOCKERFILE=f"examples/{EXAMPLE_DIR}/{IMG_NAME}.Dockerfile"
!echo $DOCKERFILE
!docker build . -t $IMAGE_URI -f $DOCKERFILE

examples/deepspeed-chat/deepspeed-chat.Dockerfile
Sending build context to Docker daemon  4.585MB
Step 1/31 : FROM us-docker.pkg.dev/vertex-ai/training/pytorch-gpu.1-13.py310:latest
 ---> 78c144ecd81c
Step 2/31 : ENV PYTHONPATH=/opt/conda/lib/python3.10/site-packages:${PYTHONPATH}
 ---> Using cache
 ---> 5cd78fb02482
Step 3/31 : RUN chmod 666 /var/log-storage/output.log
 ---> Using cache
 ---> a19250deeb80
Step 4/31 : ENV STAGE_DIR=/tmp
 ---> Using cache
 ---> 626b4bc63513
Step 5/31 : RUN mkdir -p ${STAGE_DIR}
 ---> Using cache
 ---> 9d1aef5c48c3
Step 6/31 : RUN apt-get update  && apt-get install -y --no-install-recommends  openssh-client openssh-server  dnsutils iputils-ping  net-tools  libaio-dev cmake ninja-build pdsh  && rm -rf /var/lib/apt/lists/*
 ---> Using cache
 ---> 84d3bb091c45
Step 7/31 : ENV SSH_PORT=2222
 ---> Using cache
 ---> a10ccb585bb3
Step 8/31 : COPY config/sshd_config.sed /tmp
 ---> Using cache
 ---> 70245923db33
Step 9/31 : RUN sed -i -E -f /tmp/sshd_config.sed /

In [ ]:
# If this throws error
# add "us-docker.pkg.dev": "gcloud" to /home/jupyter/.docker/config.json
!gcloud auth configure-docker

In [ ]:
# Make sure the repo specified in $AR_REPO exists.
# push the image
!echo $IMAGE_URI
!docker push $IMAGE_URI

us-central1-docker.pkg.dev/argolis-lsj-test/llama2/deepspeed-chat:1023
The push refers to repository [us-central1-docker.pkg.dev/argolis-lsj-test/llama2/deepspeed-chat]

f4415b63: Preparing 
a7522e63: Preparing 
8c4c553f: Preparing 
88d8942e: Preparing 
c2bfada6: Preparing 
73166935: Preparing 
b29cfdc5: Preparing 
6064ce0d: Preparing 
bb27861f: Preparing 
c1a7a815: Preparing 
04c8cac9: Preparing 
9b3321f2: Preparing 
7f13fbf0: Preparing 
fe2a49dd: Preparing 
64833029: Preparing 
183c9ad8: Preparing 
74c7df37: Preparing 
09d25265: Preparing 
225e8740: Preparing 
dc2fb310: Preparing 
a15e9e6d: Preparing 
b05b4a66: Preparing 
03eb5103: Preparing 
105d38de: Preparing 
29cfdc5: Waiting g 
eb8da3b6: Preparing 
3741a401: Preparing 
4314a1a9: Preparing 
e1a4db2c: Preparing 
2d93004e: Preparing 
c5d23056: Preparing 
988466f1: Preparing 
a520fb4d: Preparing 
b27861f: Waiting g 
ede7a422: Preparing 
4c8cac9: Waiting g 
c1826dae: Preparing 
475b74ad: Preparing 
8569388a: Preparing 
cc30cd40: Prep

# Test container with aiplatform.CustomJob

In [ ]:
from datetime import datetime
from google.cloud import aiplatform

aiplatform.init(project=PROJECT_ID, staging_bucket=BUCKET, location=LOCATION)

### Configure the custom job


- DATA_PATHS - dataset name in huggingface or GCS path
- MODEL_PATH - model name in huggingface or GCS path
- DATA_SPLIT - "10,40,50" means 10% of the data is used for SFT. The DeepspeedChat code converts the string into fractions (data_utils.py). We do not demo it in this notebook.
- ZERO_STAGE - ZERO stage
- PER_DEVICE_BATCH_SIZE - training batch size

The demo shows how to fine-tune Llama2-7b-hf and samsum dataset using SFT(*train_deepspeed_sft.sh*) or LoRA (*train_deepspeed_lora.sh*). To be competable with samsum dataset and users' customized datasets, the samsum interface and customized dataset interface are implemented in *third_party/utils/data/data_utils.py* and *raw_datasets.py*.

Model uploaded to Vertex AI Model Regitry is also implemented in *main.py* and *model_registry.py*. If *serving_container_image_uri* is not None, the registration process will start.

In [ ]:
# configure machine specs, and environment variables 
TOTAL_GPU_NEEDED = 8
CPU_UNIT = 12
ACCELERATOR_COUNT = 4 #per node
CPU_COUNT = CPU_UNIT * ACCELERATOR_COUNT
REPLICA_COUNT = TOTAL_GPU_NEEDED / ACCELERATOR_COUNT

ENVS = [
    {"name": "MODEL_PATH", "value": "/gcs/deepspeed_repo/base_model/Llama-2-7b-hf/Llama-2-7b-hf"},                        
    {"name": "DATA_PATHS", "value": "/gcs/deepspeed_repo/dataset/samsum"},
    {"name": "DATA_SPLIT", "value": "10,0,0"},                       
    {"name": "ZERO_STAGE", "value": "3"},
    {"name": "PER_DEVICE_BATCH_SIZE", "value": "4"},
    {"name": "NUM_GPU_PER_NODE", "value": f"{ACCELERATOR_COUNT}"},
                
]
# `WorkerPoolSpec` for worker pool 0, primary replica, required  
worker_pool_specs_1 = {
        "machine_spec": {
            "machine_type": f"g2-standard-{CPU_COUNT}",
            "accelerator_type": "NVIDIA_L4",
            "accelerator_count": ACCELERATOR_COUNT,       
        },
        "replica_count": 1,
        "container_spec": {
            "image_uri": IMAGE_URI,
            "command": [],
            "args": [],
            "env": ENVS,                
        },
        "disk_spec": {
            "boot_disk_size_gb": 1000,            
        }
    }

worker_pool_specs_2 = {
        "machine_spec": {
            "machine_type": f"g2-standard-{CPU_COUNT}",
            "accelerator_type": "NVIDIA_L4",
            "accelerator_count": ACCELERATOR_COUNT,       
        },
        "replica_count": REPLICA_COUNT - 1,
        "container_spec": {
            "image_uri": IMAGE_URI,
            "command": [],
            "args": [],
            "env": ENVS,                
        },
        "disk_spec": {
            "boot_disk_size_gb": 1000,            
        }
    }
worker_pool_specs = []
if REPLICA_COUNT == 1:
   worker_pool_specs = [worker_pool_specs_1 ]
elif REPLICA_COUNT > 1:
   worker_pool_specs = [worker_pool_specs_1, worker_pool_specs_2]


TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")
JOB_NAME  = "DeepSpeed Chat Torchrun Test " + TIMESTAMP

my_job = aiplatform.CustomJob(
    display_name=JOB_NAME,    
    worker_pool_specs=worker_pool_specs,
)

# Checking Service account that will launch the job
!gcloud config get account

703099487153-compute@developer.gserviceaccount.com


In [ ]:
#####
# Either create or reuse a tensorboard
# tensorboard = aiplatform.Tensorboard.create(
#    display_name=JOB_NAME,
# )
# 
tensorboard_name = "2069606338916253696"
tensorboard = aiplatform.Tensorboard(tensorboard_name=tensorboard_name)
# 
print(tensorboard.resource_name)

projects/703099487153/locations/us-central1/tensorboards/2069606338916253696


### Running the CustomJob

Custom Service Account - https://cloud.google.com/vertex-ai/docs/general/custom-service-account. For custom service account, be sure to first grant the SA running this notebook the "Service Account User" role, otherwise you won't be able to launch the job with the custom service account.

Tensorboard - https://cloud.google.com/vertex-ai/docs/experiments/tensorboard-training. Your training script must be configured to write TensorBoard logs to the Cloud Storage bucket, the location of which the Vertex AI Training Service will automatically make available through a predefined environment variable AIP_TENSORBOARD_LOG_DIR.

In [ ]:
my_job.submit(    
    enable_web_access=True, # For debugging
    # network="projects/{PROJECT_NUMBER}/global/networks/{PEER_NETWORK_NAME}",
    service_account="703099487153-compute@developer.gserviceaccount.com",
    tensorboard=tensorboard.resource_name,
)

Creating CustomJob
CustomJob created. Resource name: projects/703099487153/locations/us-central1/customJobs/7199000499658424320
To use this CustomJob in another session:
custom_job = aiplatform.CustomJob.get('projects/703099487153/locations/us-central1/customJobs/7199000499658424320')
View Custom Job:
https://console.cloud.google.com/ai/platform/locations/us-central1/training/7199000499658424320?project=703099487153
View Tensorboard:
https://us-central1.tensorboard.googleusercontent.com/experiment/projects+703099487153+locations+us-central1+tensorboards+2069606338916253696+experiments+7199000499658424320


While the Deepspeed Chat team has auto-tuning on roadmap, if you encounter CUDA OOM right now their advice is:
- Reduce `--per_device_*_batch_size`,
- Increase `--zero_stage {0,1,2,3}` on multi-gpu setups,
- Enable `--gradient_checkpointing` or `--only_optimize_lora`,
- Increase `--gradient_accumulate_steps {#}`, higher number reduces communication of gradients between steps

# Configurations backup

Testing facebook/opt-125m and Dahoas/synthetic-instruct-gptj-pairwise with 2 1xT4@n1-standard-4:

PER_DEVICE_BATCH_SIZE - 8 will utilize < half a T4's memory on each of the 2 nodes, 32 uses the memory 70+%

In [ ]:
#opt-125m backup
{"name": "MODEL_PATH", "value": "facebook/opt-125m"},                        
{"name": "DATA_PATHS", "value": "Dahoas/synthetic-instruct-gptj-pairwise"},                        
{"name": "DATA_SPLIT", "value": "10,40,50"},
{"name": "ZERO_STAGE", "value": "3"},
{"name": "PER_DEVICE_BATCH_SIZE", "value": "32"},

In [ ]:
# llama2 backup
{"name": "MODEL_PATH", "value": "meta-llama/Llama-2-7b-hf"},                        
{"name": "DATA_PATHS", "value": "Dahoas/rm-static Dahoas/full-hh-rlhf Dahoas/synthetic-instruct-gptj-pairwise yitingxie/rlhf-reward-datasets"},                        
{"name": "DATA_SPLIT", "value": "2,4,4"},
{"name": "ZERO_STAGE", "value": "3"},
{"name": "PER_DEVICE_BATCH_SIZE", "value": "4"}, 